In [ ]:
!git clone https://github.com/statsbomb/open-data.git

In [ ]:
import json
import os
import altair as alt
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
with open('open-data/data/matches/72/30.json', 'r') as fo:
  j = fo.read()
matches = json_normalize(json.loads(j), meta='match_id')
# There should be 52 World Cup matches
assert len(matches) == 52, 'Did not load all matches! Only loaded {} matches.'.format(len(matches))

In [ ]:
match_ids = matches.loc[:, 'match_id'].tolist()
df = pd.DataFrame()
for match_id in match_ids:
  fo = open('open-data/data/events/{}.json'.format(match_id), 'r')
  j = fo.read()
  fo.close()
  next_df = json_normalize(json.loads(j), meta='type', sep='_')
  df = pd.concat([df, next_df])

In [ ]:
USA = 1214

usa_df = (
    df.loc[
        # Rows
        (df['duel_outcome_name'].isin(['Success In Play', 'Won', 'Success Out'])) &
        (df['team_id'] == USA),
        # Columns
        ['player_name', 'location']
    ]
)

usa_df['player_last_name'] = [n.split()[-1:][0] for n in usa_df['player_name']]

print(usa_df.shape)
print(usa_df.head())
print()

usa_legend_df = (
    usa_df.loc[:, ['player_name', 'player_last_name']]
        .groupby(['player_last_name'])
        .count()
        .reset_index()
)
usa_legend_df.columns = ['player_last_name', 'count']

print(usa_legend_df.shape)
print(usa_legend_df.head())

(87, 3)
              player_name      location player_last_name
171          Rose Lavelle  [81.6, 22.5]          Lavelle
269  Crystal Alyssia Dunn    [5.5, 2.3]             Dunn
628        Samantha Mewis   [58.0, 4.5]            Mewis
896           Tobin Heath  [77.6, 62.6]            Heath
948  Crystal Alyssia Dunn   [79.3, 3.4]             Dunn

(17, 2)
  player_last_name  count
0            Brian      1
1       Dahlkemper      3
2         Davidson      2
3             Dunn     14
4             Ertz      8


In [ ]:
x_vals = alt.X('location[0]:Q', scale=alt.Scale(domain=[0, 120]), title=None)
y_vals = alt.Y('location[1]:Q', scale=alt.Scale(domain=[0, 80]), title=None)

selector = alt.selection_multi(fields=['player_last_name'])
color = alt.condition(selector,
                      alt.Color('player_last_name:N', legend=None),
                      alt.value('#FFFFFF00'))

pitch = (
    alt.Chart(usa_df)
        .properties(
            width=600,
            height=400,
            description='USA duels won')
        .mark_point()
        .encode(
            x=x_vals,
            y=y_vals,
            color=color)
)

lbls = (
    pitch
        .mark_text(
            align='left',
            baseline='middle',
            dx=7)
        .encode(
            text='player_last_name')
)

legend = (
    alt.Chart(usa_legend_df)
        .mark_point()
        .encode(
            y=alt.Y(
                shorthand='player_last_name:N',
                axis=alt.Axis(orient='right'),
                title='Click circle next to player name to highlight on chart',
                sort=alt.EncodingSortField(field='count', order='descending')),
            color=color)
        .add_selection(selector)
)

pitch + lbls | legend

HConcatChart({
  hconcat: [LayerChart({
    data:                player_name       location player_last_name
    171           Rose Lavelle   [81.6, 22.5]          Lavelle
    269   Crystal Alyssia Dunn     [5.5, 2.3]             Dunn
    628         Samantha Mewis    [58.0, 4.5]            Mewis
    896            Tobin Heath   [77.6, 62.6]            Heath
    948   Crystal Alyssia Dunn    [79.3, 3.4]             Dunn
    1459        Samantha Mewis   [29.4, 19.8]            Mewis
    1744        Samantha Mewis    [27.8, 9.9]            Mewis
    1777         Kelley O'Hara  [100.2, 78.1]           O'Hara
    1860      Becky Sauerbrunn   [61.5, 46.0]       Sauerbrunn
    2145         Kelley O'Hara    [4.0, 63.2]           O'Hara
    2450      Becky Sauerbrunn    [53.3, 9.4]       Sauerbrunn
    2881         Lindsey Horan   [24.6, 56.0]            Horan
    2976           Tobin Heath   [91.9, 45.3]            Heath
    3334      Becky Sauerbrunn   [55.9, 50.0]       Sauerbrunn
    3709  Crystal Alyssia Dunn    [24.5, 4.1]             Dunn
    3773        Christen Press    [8.3, 10.4]            Press
    317   Crystal Alyssia Dunn   [68.8, 12.9]             Dunn
    550          Kelley O'Hara   [79.2, 64.6]           O'Hara
    1004           Alex Morgan   [49.3, 56.7]           Morgan
    1886            Julie Ertz    [59.3, 9.6]             Ertz
    2227           Carli Lloyd  [117.1, 60.7]            Lloyd
    2266            Julie Ertz    [9.5, 62.2]             Ertz
    2419         Lindsey Horan   [55.3, 59.8]            Horan
    2729       Abby Dahlkemper   [39.6, 44.7]       Dahlkemper
    2819           Alex Morgan   [64.8, 42.0]           Morgan
    3121           Alex Morgan   [52.8, 59.6]           Morgan
    3358       Abby Dahlkemper   [73.8, 75.2]       Dahlkemper
    184           Morgan Brian   [94.7, 60.6]            Brian
    354        Tierna Davidson   [85.1, 32.7]         Davidson
    386         Christen Press   [79.5, 10.3]            Press
    ...                    ...            ...              ...
    2597      Becky Sauerbrunn   [52.3, 32.6]       Sauerbrunn
    2630      Becky Sauerbrunn    [11.5, 4.0]       Sauerbrunn
    2669  Crystal Alyssia Dunn   [14.5, 10.3]             Dunn
    3034  Crystal Alyssia Dunn   [27.4, 12.7]             Dunn
    3154           Tobin Heath  [115.5, 72.8]            Heath
    59           Kelley O'Hara   [28.1, 72.5]           O'Hara
    126   Crystal Alyssia Dunn    [36.4, 4.9]             Dunn
    456          Lindsey Horan    [63.7, 6.2]            Horan
    1228  Crystal Alyssia Dunn    [6.2, 12.7]             Dunn
    1711            Julie Ertz   [57.1, 43.0]             Ertz
    1931  Crystal Alyssia Dunn    [54.9, 3.3]             Dunn
    2882           Carli Lloyd   [76.9, 24.4]            Lloyd
    168         Samantha Mewis     [6.8, 6.5]            Mewis
    461         Samantha Mewis   [76.7, 22.6]            Mewis
    740   Crystal Alyssia Dunn   [25.3, 38.3]             Dunn
    1272          Rose Lavelle   [36.4, 68.8]          Lavelle
    1635        Samantha Mewis   [73.3, 15.0]            Mewis
    1676        Samantha Mewis    [33.5, 9.2]            Mewis
    1780           Ali Krieger   [11.1, 72.0]          Krieger
    1897  Crystal Alyssia Dunn   [22.3, 17.4]             Dunn
    2010           Tobin Heath   [28.5, 57.3]            Heath
    2074          Rose Lavelle   [41.4, 54.7]          Lavelle
    2107        Samantha Mewis    [57.2, 5.6]            Mewis
    2131            Julie Ertz   [11.6, 49.8]             Ertz
    2246            Julie Ertz   [27.4, 27.7]             Ertz
    2287  Crystal Alyssia Dunn   [64.6, 17.8]             Dunn
    2316        Samantha Mewis   [63.9, 35.0]            Mewis
    2867        Christen Press   [33.3, 12.6]            Press
    2887           Carli Lloyd   [35.6, 15.9]            Lloyd
    3037          Rose Lavelle   [59.5, 58.8]          Lavelle
    
    [87 rows x 3 columns],
    layer: [Char